In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
'''
To create a Django web application that serves your AI model, follow the steps below. This example will allow you to create a website that receives user input (college, branch, and category) and returns the predicted cutoff for 2024.

### Step 1: Install Django and Joblib

First, create a virtual environment and install Django along with Joblib.

```bash
# Create a virtual environment (if not already created)
python -m venv env
source env/bin/activate  # On Windows, use `env\Scripts\activate`

# Install Django and other dependencies
pip install django joblib pandas scikit-learn
```

### Step 2: Create a New Django Project

Create a new Django project and app.

```bash
# Create a new Django project
django-admin startproject cutoff_predictor

# Navigate to the project folder
cd cutoff_predictor

# Create a new Django app
python manage.py startapp prediction
```

### Step 3: Save the Model

Save the trained model using Joblib as described earlier:

```python
import joblib

# Save the trained model
joblib.dump(model, 'model/cutoff_predictor_model.pkl')
```

Ensure the model is stored in the `model/` directory inside your Django project.

### Step 4: Set Up Django App

In the `cutoff_predictor/settings.py` file, add the newly created `prediction` app to the `INSTALLED_APPS` list.

```python
INSTALLED_APPS = [
    # Other apps...
    'prediction',
]
```

### Step 5: Create the Views

In the `prediction/views.py` file, write a view to handle the form submission and prediction:

```python
from django.shortcuts import render
import joblib
import pandas as pd

# Load the model
model = joblib.load('model/cutoff_predictor_model.pkl')

def index(request):
    # Define options for college, branch, and category
    colleges = ['College 1', 'College 2', 'College 3']
    branches = ['Branch 1', 'Branch 2']
    categories = ['Category 1', 'Category 2']

    if request.method == 'POST':
        # Get the form data
        college = request.POST['college']
        branch = request.POST['branch']
        category = request.POST['category']
        
        # Prepare input data in the expected format
        input_data = pd.DataFrame([[college, branch, category, 0, 0, 0, 0, 0]],
                                  columns=['COLLEGE', 'BRANCH', 'CATEGORIES', 
                                           'Cutoff_2019', 'Cutoff_2020', 'Cutoff_2021', 'Cutoff_2022', 'Cutoff_2023'])

        # Make prediction
        prediction = model.predict(input_data)

        return render(request, 'index.html', {'prediction': prediction[0], 'colleges': colleges, 'branches': branches, 'categories': categories})

    return render(request, 'index.html', {'prediction': None, 'colleges': colleges, 'branches': branches, 'categories': categories})
```

### Step 6: Define URL Patterns

In the `prediction/urls.py` file, define the URL for the `index` view:

```python
from django.urls import path
from . import views

urlpatterns = [
    path('', views.index, name='index'),
]
```

Make sure to include this app's URL patterns in the `cutoff_predictor/urls.py` file:

```python
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('prediction.urls')),
]
```

### Step 7: Create HTML Template

In the `prediction/templates` directory, create an HTML template `index.html` that will display the form and the prediction result.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="{% static 'style.css' %}">
    <title>Cutoff Predictor</title>
</head>
<body>
    <header>
        <h1>2025 Cutoff Ranking Prediction</h1>
    </header>
    <div class="container">
        <form method="POST">
            {% csrf_token %}
            <label for="college">Select College</label>
            <select name="college" id="college" required>
                <option value="">Select One</option>
                {% for college in colleges %}
                <option value="{{ college }}">{{ college }}</option>
                {% endfor %}
            </select>

            <label for="branch">Select Branch</label>
            <select name="branch" id="branch" required>
                <option value="">Select One</option>
                {% for branch in branches %}
                <option value="{{ branch }}">{{ branch }}</option>
                {% endfor %}
            </select>

            <label for="category">Select Category</label>
            <select name="category" id="category" required>
                <option value="">Select One</option>
                {% for category in categories %}
                <option value="{{ category }}">{{ category }}</option>
                {% endfor %}
            </select>

            <input type="submit" value="Predict">
        </form>

        {% if prediction is not none %}
        <div class="result">
            <h2>Predicted Cutoff for 2024: {{ prediction }}</h2>
        </div>
        {% endif %}
    </div>
</body>
</html>
```

### Step 8: Create Static Files (CSS)

Create a `static` folder in your project and add a `style.css` file to style the page.

```css
body {
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
    background-color: #f4f4f4;
}

header {
    background-color: #333;
    color: white;
    padding: 1rem;
    text-align: center;
}

.container {
    max-width: 600px;
    margin: 20px auto;
    padding: 20px;
    background-color: white;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
}

form label {
    display: block;
    margin: 10px 0 5px;
}

form select,
form input[type="submit"] {
    width: 100%;
    padding: 10px;
    margin: 10px 0;
    border-radius: 5px;
    border: 1px solid #ccc;
}

form input[type="submit"] {
    background-color: #4CAF50;
    color: white;
    cursor: pointer;
}

form input[type="submit"]:hover {
    background-color: #45a049;
}

.result {
    margin-top: 20px;
    padding: 10px;
    background-color: #f1f1f1;
    text-align: center;
}
```

### Step 9: Run the Django Application

Now that everything is set up, run the Django development server to test your application:

```bash
# Run the Django development server
python manage.py runserver
```

### Step 10: Access the Web Application

Visit `http://127.0.0.1:8000/` in your browser, where you should see a form to select the college, branch, and category. After submitting the form, the predicted cutoff for 2024 will be displayed.

### Step 11: Deploying the Application

Once you are satisfied with your local application, you can deploy it to a platform like **Heroku**, **AWS**, or **Google Cloud**. Django applications are easy to deploy, and there are various guides for deploying to those services.

#### For Heroku:

1. Install the `gunicorn` web server:
   ```bash
   pip install gunicorn
   ```

2. Create a `Procfile` in the root directory of your project:
   ```
   web: gunicorn cutoff_predictor.wsgi
   ```

3. Push your code to a Git repository and follow the Heroku deployment steps.

---

### Summary

You’ve now created a Django web application that:
- Loads your trained AI model using **Joblib**.
- Accepts user input via an HTML form.
- Displays the predicted cutoff for 2024 based on the user’s selection.
'''

'\nTo create a Django web application that serves your AI model, follow the steps below. This example will allow you to create a website that receives user input (college, branch, and category) and returns the predicted cutoff for 2024.\n\n### Step 1: Install Django and Joblib\n\nFirst, create a virtual environment and install Django along with Joblib.\n\n```bash\n# Create a virtual environment (if not already created)\npython -m venv env\nsource env/bin/activate  # On Windows, use `env\\Scripts\x07ctivate`\n\n# Install Django and other dependencies\npip install django joblib pandas scikit-learn\n```\n\n### Step 2: Create a New Django Project\n\nCreate a new Django project and app.\n\n```bash\n# Create a new Django project\ndjango-admin startproject cutoff_predictor\n\n# Navigate to the project folder\ncd cutoff_predictor\n\n# Create a new Django app\npython manage.py startapp prediction\n```\n\n### Step 3: Save the Model\n\nSave the trained model using Joblib as described earlier:\n\n

In [4]:
import os

In [5]:
os.listdir()

['.ipynb_checkpoints', 'Untitled.ipynb']

In [9]:
os.listdir()

['model', '__init__.py', 'notebook', 'data']

In [11]:
df=pd.read_csv('data/comple_n1.csv')

In [12]:
df.head()

,COLLEGE,BRANCH,CATEGORIES,Cutoff_2019,Cutoff_2020,Cutoff_2021,Cutoff_2022,Cutoff_2023,Cutoff_2024
0,University of Visvesvaraya College of Engineering,AI Artificial Intelligence,2AG,0,0,0,371,432.0,475
1,University of Visvesvaraya College of Engineering,AI Artificial Intelligence,GM,0,0,0,298,249.0,355
2,University of Visvesvaraya College of Engineering,AI Artificial Intelligence,SCG,0,0,0,815,1733.0,670
3,University of Visvesvaraya College of Engineering,AI Artificial Intelligence,GMR,0,0,0,1201,0.0,0
4,University of Visvesvaraya College of Engineering,AI Artificial Intelligence,SCR,0,0,0,0,0.0,0


In [20]:
college_names=df.COLLEGE.unique()
Branch_names=df.BRANCH.unique()
Categories_names=df.CATEGORIES.unique()

In [23]:
df[(df["COLLEGE"]=="University of Visvesvaraya College of Engineering")&(df["BRANCH"]=="AI Artificial Intelligence")&(df["CATEGORIES"]=="2AG")]

,COLLEGE,BRANCH,CATEGORIES,Cutoff_2019,Cutoff_2020,Cutoff_2021,Cutoff_2022,Cutoff_2023,Cutoff_2024
0,University of Visvesvaraya College of Engineering,AI Artificial Intelligence,2AG,0,0,0,371,432.0,475


In [20]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# Load data from CSV file
df = pd.read_csv('src/data/comple_n1.csv')

# Handle NaN values by filling with a default value (e.g., 0) or using SimpleImputer
imputer = SimpleImputer(strategy='mean')
df[['Cutoff_2019', 'Cutoff_2020', 'Cutoff_2021', 'Cutoff_2022', 'Cutoff_2023']] = imputer.fit_transform(
    df[['Cutoff_2019', 'Cutoff_2020', 'Cutoff_2021', 'Cutoff_2022', 'Cutoff_2023']]
)

# Prepare features and target
X = df[['COLLEGE', 'BRANCH', 'CATEGORIES', 'Cutoff_2019', 'Cutoff_2020', 'Cutoff_2021', 'Cutoff_2022', 'Cutoff_2023']]
y = df['Cutoff_2024']

# Define the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['COLLEGE', 'BRANCH', 'CATEGORIES']),
    ],
    remainder='passthrough'  # Keep other columns (the cutoff columns)
)

# Create a pipeline that combines the preprocessing with the Ridge model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

# Fit the model
model.fit(X, y)

/home/ibrahimfadu/anaconda3/lib/python3.11/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['COLLEGE', 'BRANCH',
                                                   'CATEGORIES'])])),
                ('regressor', Ridge())])

In [ ]:
pro